In [67]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [76]:
#wczytanie danych
data = pd.read_excel("dane_do_wczytania.xlsx")
#ustawiamy kolumnę "wojewodztwo" jako index
#inplace powoduje że modyfikujemy DataFrame, a nie tworzymy nowy
data.set_index("wojewodztwo",inplace=True)
#usuwamy nazwę indexu
data.index.names = [None]
#data

In [77]:
#zamiana zmiennych na stymulanty
print(data.columns)
#liczba lekarzy - stymulanta
#liczba pielegniarek - stymulanta
#liczba aptek - stymulanta
#zgony ogolem - destymulanta
#zespoly ratownictwa medycznego - stymulanta
#liczba ludnosci na lozko szpitalne - destymulanta
#absolwenci - stymulanta
#organizacje non profit - nominanta, 8%

Index(['lekarze [liczba/tys ludnosci]', 'pielegniarki [liczba/tys ludnosci]',
       'apteki [liczba/km2]',
       'zgony ogolem na tys urodzen zywych [liczba/tys urodzen zywych]',
       'zespoly ratownictwa medycznego [liczba/tys ludnosci]',
       'liczba ludnosci na lozko w szpitalach ogolnych [osoby/lozko]',
       'absolwenci uczelni medycznych [osoba]',
       'organizacje non profit ochrona zdrowia [%]'],
      dtype='object')


In [78]:
data["liczba ludnosci na lozko w szpitalach ogolnych [osoby/lozko]"] = data["liczba ludnosci na lozko w szpitalach ogolnych [osoby/lozko]"] *-1
data["zgony ogolem na tys urodzen zywych [liczba/tys urodzen zywych]"] = data["zgony ogolem na tys urodzen zywych [liczba/tys urodzen zywych]"] * -1
nominanta = 9
for i in range(len(data["organizacje non profit ochrona zdrowia [%]"])):
    if data["organizacje non profit ochrona zdrowia [%]"][i]==nominanta :
        data["organizacje non profit ochrona zdrowia [%]"][i]=1
    elif data["organizacje non profit ochrona zdrowia [%]"][i]>nominanta :
        data["organizacje non profit ochrona zdrowia [%]"][i] = 1/(data["organizacje non profit ochrona zdrowia [%]"][i]-nominanta+1)
    else:
        data["organizacje non profit ochrona zdrowia [%]"][i] = -1/(data["organizacje non profit ochrona zdrowia [%]"][i]-nominanta-1)


In [174]:
#standaryzacja danych I
data_std = data.copy()
col_names = data_std.columns
for column in col_names:
    mean = np.mean(data_std[column])
    sd = np.std(data_std[column])
    for i in range(len(data_std[column])):
        data_std[column][i] = (data_std[column][i] - mean)/sd

In [80]:
#standaryzacja danych II (dla TOPSIS)
data_std_TOPSIS = data.copy()
col_names = data_std_TOPSIS.columns
for column in col_names:
    suma = np.sum(data_std_TOPSIS[column]**2)
    for i in range(len(data_std_TOPSIS[column])):
        data_std_TOPSIS[column][i] = data_std_TOPSIS[column][i]/np.sqrt(suma)

In [175]:
#metoda standaryzowanych sum

#przypisanie wag
weights = pd.DataFrame({"Name":col_names, "Weights":[0.25,0.1,0.1,0.2,0.075,0.075,0.15,0.05,]})
weights = weights.transpose()
weights.rename(columns=weights.iloc[0],inplace=True)
weights.drop(weights.index[0],inplace=True)
#np.sum(weights["Weights"])

#miara syntetyczna
S1 = np.empty(len(data_std.index))
for i in range(len(data_std.index)):
    suma_czesciowa = 0
    for column in data_std.columns:
        suma_czesciowa += data_std[column][i]*weights[column][0]
    S1[i] = suma_czesciowa

In [184]:
#wyliczenie i dołączenie standaryzowanej miary syntetycznej do pierwotnej tabeli
miara_synt = pd.DataFrame(S1)
miara_synt.rename(columns={0:"S1"},inplace=True)
min_s1 = min(miara_synt["S1"])
s1_min_diff = miara_synt["S1"] - min_s1
miara_synt_std = s1_min_diff/np.max(s1_min_diff)

standaryzowane_sumy = data_std.copy()
standaryzowane_sumy.insert(len(standaryzowane_sumy.columns),"S1",miara_synt["S1"].values)
standaryzowane_sumy.insert(len(standaryzowane_sumy.columns),"S2",miara_synt_std.values)

standaryzowane_sumy.sort_values("S2",inplace=True,ascending=False)
standaryzowane_sumy.drop(columns=standaryzowane_sumy.columns[range(9)])


,S2
MAZOWIECKIE,1.000000
SLASKIE,0.886398
PODLASKIE,0.821959
LUBELSKIE,0.778469
LODZKIE,0.777691
DOLNOSLASKIE,0.751242
MALOPOLSKIE,0.736129
POMORSKIE,0.587049
WIELKOPOLSKIE,0.547890
PODKARPACKIE,0.401121
